In [1]:
import numpy as np
from io import StringIO
import time,os, importlib
from tqdm import tqdm
np.set_printoptions(linewidth=160)
import lib.ktn_io as kio
import lib.gt_tools as gt
from scipy.sparse import save_npz,load_npz, diags, eye, csr_matrix,bmat,find
from scipy.sparse.linalg import eigs,inv,spsolve
from scipy.sparse.csgraph import connected_components
import scipy as sp
import scipy.linalg as spla
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.colors import LogNorm

# Load Data

In [2]:
data_path = "KTN_data/LJ38/4k"

beta = 1.0/0.15

Nmax = None

B, K, D, N, u, s, Emin, index_sel = kio.load_mat(path=data_path,beta=beta,Emax=None,Nmax=Nmax,screen=False)
D = np.ravel(K.sum(axis=0))
BF = beta*u-s

BF -= BF.min()

A_states,B_states = kio.load_AB(data_path,index_sel)

basins = B_states + A_states

inter_region = ~basins

names = ["oct","ico"]

rB,rD,GT_Q,rN,retry = gt.gt_seq(N=N,rm_reg=inter_region,B=B.copy(),D=D.copy(),trmb=10,retK=True,Ndense=50,screen=True)

r_A_states = A_states[~inter_region]
r_B_states = B_states[~inter_region]
r_BF = BF[~inter_region]

tau = np.zeros(2)
rho = np.exp(-r_BF) * r_B_states / (np.exp(-r_BF) * r_B_states).sum()
x = spsolve(GT_Q[:,~r_A_states][~r_A_states,:],rho[~r_A_states])
print(x.sum())

#GT_Q = (1-rB)@diags(rD)
# iGT_Q = 

tau = np.zeros(2)
rho = np.exp(-r_BF) * r_B_states / (np.exp(-r_BF) * r_B_states).sum()
x = spsolve(eye((~r_A_states).sum())-rB[:,~r_A_states][~r_A_states,:],rho[~r_A_states])
tau_AB = (diags(1.0/rD[~r_A_states])@x).sum()
       
       


print(GT_Q.shape,r_B_states.shape)

print("\n\tN_states:",N,"N_transitions:",K.data.size)
print("\n\t%d %s STATES <-> %d %s STATES" % (A_states.sum(),names[0],B_states.sum(),names[1]))
print("\n\tbeta:",beta,"\n\n")




  7%|▋         | 50/767 [00:00<00:00, 740.31it/s] 

N,N_TS: 4000 2653
GT regularization removing 767 states:


casting to csr_matrix
GT done, 0 rescans due to LinAlgError
29649191.509970684
(133, 133) (133,)

	N_states: 900 N_transitions: 2284

	5 oct STATES <-> 128 ico STATES

	beta: 6.666666666666667 




In [3]:
print(B.shape,N)
rBB,rDD,iG,rN,succ = gt.gt_seq(N=N,rm_reg=inter_region,B=B.copy(),DD=csr_matrix(diags(1.0/D)),trmb=10,retK=True,Ndense=50,screen=True)

print(rBB.shape,rDD.shape)

x = spsolve(iG[:,~r_A_states][~r_A_states,:],rho[~r_A_states])

print((rDD[:,~r_A_states]@x).sum()/tau_AB)



 13%|█▎        | 100/767 [00:00<00:01, 527.35it/s]

(900, 900) 900
GT regularization removing 767 states:


casting to csr_matrix
GT done, 0 rescans due to LinAlgError
(133, 133) (900, 900)
0.9980877904580229


# remove all single states

In [ ]:
Q = diags(D)-B@diags(D)

oB=B.copy()
oB.data[oB.data<1.0] = 0.0
oB.eliminate_zeros()

rm_reg = np.zeros(N,bool)
#rm_reg[find(oB)[1]] = True 
r_B,r_D,GT_Q,r_N,retries = gt.gt_seq(N=N,rm_reg=rm_reg,B=B,D=D,trmb=50,retK=True,Ndense=50,screen=True)

r_A_states = A_states[~rm_reg]
r_B_states = B_states[~rm_reg]
r_BF = BF[~rm_reg]


find_time = np.ones(r_N,bool)
find_time[r_B_states] = False
tau_b = np.zeros(find_time.sum())



GT_QB = GT_Q[:,~r_B_states][~r_B_states,:]
GT_QA = GT_Q[:,~r_A_states][~r_A_states,:]

rho_AnB = (np.exp(-r_BF) * r_A_states)[~r_B_states]
rho_AnB /= rho_AnB.sum()
rho_BnA = (np.exp(-r_BF) * r_B_states)[~r_A_states]
rho_BnA /= rho_BnA.sum()

taus = np.zeros((r_N,4))

for i,ii in tqdm(enumerate(np.arange(r_N)[~r_B_states]),total=(~r_B_states).sum()):
    select = np.arange((~r_B_states).sum())==i
    
    taus[ii][0] = spsolve(GT_QB,select).sum() # time to B
    
    # tau = k_u. Q^-2 rho
    GT_QI = GT_QB[~select,:][:,~select]
    ku = GT_QB[select,:][:,~select]
    x = spsolve(GT_QI,rho_AnB[~select])
    y = spsolve(GT_QI,x)
    taus[ii][1] = -ku@y
    

for i,ii in tqdm(enumerate(np.arange(r_N)[~r_A_states]),total=(~r_A_states).sum()):
    
    select = np.arange((~r_A_states).sum())==i
    
    taus[ii][2] = spsolve(GT_QA,select).sum() # time to B
    
    # tau = k_u. Q^-2 rho
    GT_QI = GT_QA[~select,:][:,~select]
    ku = GT_QA[select,:][:,~select]
    x = spsolve(GT_QI,rho_BnA[~select])
    y = spsolve(GT_QI,x)
    taus[ii][3] = -ku@y

    

In [ ]:
tau_AB = np.zeros(4)
rho = np.exp(-r_BF) * r_A_states
rho /= rho.sum()
x = spla.solve(GT_Q[:,~r_B_states][~r_B_states,:].todense(),rho[~r_B_states])
y = spla.solve(GT_Q[:,~r_B_states][~r_B_states,:].todense(),x)
tau_AB[2] = x.sum()
tau_AB[3] = y.sum()

print(tau_AB[-2:])
rho = np.exp(-r_BF) * r_B_states
rho /= rho.sum()
x = spla.solve(GT_Q[:,~r_A_states][~r_A_states,:].todense(),rho[~r_A_states])
y = spla.solve(GT_Q[:,~r_A_states][~r_A_states,:].todense(),x)

tau_AB[0] = x.sum()
tau_AB[1] = y.sum()

print(tau_AB[:2])


In [ ]:


"""
P(first i at t) = exp(-Q)

B_AI G_I B_IB

\sum_p \sum_i P^p_i+1i tau_i = \langle\tau_i\rangle N_i

\sum_p \sum_ij tau_itau_j P^p_i+1i)= \sum_p (\sum_i \tau_i\tau_j[(G-1)]_ij P_i + \sum_j \tau (G-1)\tau P_j + \sum_i \tau^2_i


GAUSS-JORDAN ELIMINATION METHOD

\langle\tau_i\rangle N_i

P(in j m after i) = [B^m]_ji

"""

pi = np.exp(-r_BF[Is]) / np.exp(-r_BF[Is]).sum()
print(taus[Is,2]@pi / 1.0e6)

In [ ]:
plt.figure(figsize=(16,6),dpi=120)
Is = ~r_A_states+~r_B_states

pi = np.exp(-r_BF[Is]) / np.exp(-r_BF).sum()


plt.semilogy(np.arange(r_N)[~r_B_states],taus[~r_B_states,1]+taus[~r_B_states,0],'C0o')
plt.semilogy(np.arange(r_N)[~r_A_states],taus[~r_A_states,3]+taus[~r_A_states,2],'C1s')
plt.semilogy(np.arange(r_N)[r_B_states],taus[r_B_states,-2:].sum(axis=1),'rs',markersize=10)
plt.semilogy(np.arange(r_N)[r_A_states],taus[r_A_states,:2].sum(axis=1),'bo',markersize=10)





#plt.semilogy(1.0/tau_b[r_A_states[~r_B_states]],'D')
plt.semilogy(np.arange(r_N),np.ones(r_N)*tau_AB[1],'k--')
plt.semilogy(np.arange(r_N),np.ones(r_N)*tau_AB[0],'k--')





In [ ]:
rBB = r_B.copy()[~r_B_states,:][:,~r_B_states]
fi = find(rBB) # i,j,data
nd = np.zeros(fi[2].size)
nd = taus[~r_B_states][fi[0],:2].sum(axis=1)/taus[~r_B_states][fi[1],:2].sum(axis=1)

time_to_M = csr_matrix((nd,(fi[0],fi[1])),shape=(tau_b.size,tau_b.size))

rat = 1.1
time_to_M -= diags(time_to_M.diagonal())
time_to_M.data[time_to_M.data<1.0/rat] = 0.0
time_to_M.data[time_to_M.data>rat] = 0.0
time_to_M.eliminate_zeros()

nc,cc = connected_components(time_to_M)
print(np.bincount(cc),nc)

fig = plt.figure(figsize=(8,8),dpi=100)
#ax = fig.add_subplot(111)
#ax.matshow(time_to_M.todense())#,cmap)='hot_r')
plt.spy(time_to_M,markersize=0.3)
plt.show()


- calculate time to target for whole system - one large task but O(N) and can be arbitrarily good
- group according to uniformity of time and connection then make threshold on ratio
- can we be variational about this? Can we produce an error bound under this criteria?
# This is the only thing really worth pushing to lower T for

In [ ]:
Q = diags(D)-B@diags(D)

rho = np.exp(-BF) * A_states
rho /= rho.sum()

ftau=[0.,0.]
ftau[0] = spsolve(Q[:,~B_states][~B_states,:],rho[~B_states]).sum()

rho = np.exp(-BF) * B_states
rho /= rho.sum()
ftau[1] = spsolve(Q[:,~A_states][~A_states,:],rho[~A_states]).sum()


oB=B.copy()
oB.data[oB.data<1.0] = 0.0
oB.eliminate_zeros()

rm_reg = np.zeros(N,bool)
rm_reg[find(oB)[1]] = True 
r_B,r_D,GT_Q,r_N,retries = gt.gt_seq(N=N,rm_reg=rm_reg,B=B,D=D,trmb=10,retK=True,Ndense=50,screen=True)

r_A_states = A_states[~rm_reg]
r_B_states = B_states[~rm_reg]
r_BF = BF[~rm_reg]
print(GT_Q.shape,r_B_states.shape)


rho = np.exp(-r_BF) * r_A_states
rho /= rho.sum()
print("tau:",spsolve(GT_Q[:,~r_B_states][~r_B_states,:],rho[~r_B_states]).sum()/ftau[0],end=" ")
rho = np.exp(-r_BF) * r_B_states
rho /= rho.sum()
print("tau:",spsolve(GT_Q[:,~r_A_states][~r_A_states,:],rho[~r_A_states]).sum()/ftau[1],end=" ")





- Kemeny constant for nonabsorbing chain gives a mixing time. For an abosrbing chain it thus gives mixing time conditional on not absorbing.
- If Kemeny/tau is small, we thus mix before exit

- Recursive clustering

- define max size

- Local Kemeny constant for basin


In [ ]:
def bp_regroup(_B,_D,_BF,thresh=0.5,fs=None,dogt=False):
    lBF = _BF.copy()
    T = _B.copy()
    T = _B.minimum(_B.transpose())
    T = T@diags(T.indptr[1:]-T.indptr[:-1])
    T.data[T.data<thresh] = 0.0
    #T = T.minimum(T.transpose())
    T.eliminate_zeros()
    cc = connected_components(T)[1]
    if dogt:
        return cc_gt(_B,_D,_BF,cc,fs)
    else:
        pop = np.bincount(cc)
        print(pop.size,pop.mean(),pop.max())
    
def fe_regroup(_B,_D,_BF,thresh=0.5,fs=None,dogt=False):
    
    T = _B.copy()@diags(_D.copy())
    T = T.minimum(T.transpose())
    #T = T@diags(T.indptr[1:]-T.indptr[:-1])
    T.data[T.data<np.exp(-thresh)] = 0.0
    #T = T.minimum(T.transpose())
    T.eliminate_zeros()
    cc = connected_components(T)[1]
    if dogt:
        return cc_gt(_B,_D,_BF,cc,fs)
    else:
        pop = np.bincount(cc)
        print(pop.size,pop.mean(),pop.max())
    
    
def cc_gt(_B,_D,_BF,_cc,_fs):
    lBF = _BF.copy()
    rm_reg = np.zeros(_BF.size,bool)
    
    select = np.bincount(_cc)>1
    if not _fs is None:
        for i in range(select.size):
            select[i] *= (((_cc==i)*_fs).sum()==0)
    
    for i in select.nonzero()[0]:
        mbf = np.ones(_BF.size)*_BF.max()
        mbf[_cc==i] = lBF[_cc==i]
        rm_reg += _cc==i
        rm_reg[mbf.argmin()] = False
        lBF[mbf.argmin()] = -np.log(np.exp(_BF[_cc==i]).sum())
    rm_reg = rm_reg.astype(bool)
    lB,lD,lN,ret = gt.gt_seq(N=_BF.size,rm_reg=rm_reg,B=_B,D=_D,trmb=3,retK=False,Ndense=50,screen=False)
    
    
    return lB,lD,lN,lBF[~rm_reg],rm_reg

def kemeny_regroup(_Q,thresh=1.0):
    T = _Q.minimum(_Q.transpose())
    T.data[T.data<np.exp(-thresh)] = 0.0
    T.eliminate_zeros()
    nc,cc = connected_components(T)    
    pop = np.bincount(cc)
    

In [ ]:
ABS = r_A_states + r_B_states
for thresh in np.logspace(-10,-1,10):#linspace(1.0,18.0,17):
    print(thresh,end=": ")
    bp_regroup(r_B,r_D,r_BF,thresh=thresh,fs=ABS)

print("------------")

for thresh in np.linspace(1.0,18.0,17):
    print(thresh,end=": ")
    fe_regroup(r_B,r_D,r_BF,thresh=thresh,fs=ABS)


In [ ]:
print(-np.log((B@diags(D)).data).mean())


Q = diags(D)-B@diags(D)

rho = np.exp(-BF) * A_states
rho /= rho.sum()

ftau=[0.,0.]
ftau[0] = spsolve(Q[:,~B_states][~B_states,:],rho[~B_states]).sum()

rho = np.exp(-BF) * B_states
rho /= rho.sum()
ftau[1] = spsolve(Q[:,~A_states][~A_states,:],rho[~A_states]).sum()

ABS = A_states.copy()+B_states.copy()

for thresh in np.linspace(.0,18.0,17):
    lBDNF = fe_regroup(B,D,BF,thresh=thresh,fs=ABS)
    print(thresh,lBDNF[0].shape,end=" ")
    r_A_states = A_states[~lBDNF[-1]]
    r_B_states = B_states[~lBDNF[-1]]


    rho = np.exp(-lBDNF[-2]) * r_A_states
    rho /= rho.sum()
    r_Q = (eye(lBDNF[1].size)-lBDNF[0])@diags(lBDNF[1])
    print("tau:",spsolve(r_Q[:,~r_B_states][~r_B_states,:],rho[~r_B_states]).sum()/ftau[0],end=" ")
    rho = np.exp(-lBDNF[-2]) * r_B_states
    rho /= rho.sum()
    r_Q = (eye(lBDNF[1].size)-lBDNF[0])@diags(lBDNF[1])
    print("tau:",spsolve(r_Q[:,~r_A_states][~r_A_states,:],rho[~r_A_states]).sum()/ftau[1],"\n")
    


In [ ]:
def split_and_connect(_T,thresh=1.0,min_block=10):
    T = _T.copy()
    T = T.maximum(T.transpose())
    T.data[T.data>thresh] = 0.0
    T.eliminate_zeros()
    nc,cc = connected_components(T)
    
    pop = np.bincount(cc)
    
    cid = np.arange(nc)[pop>=min_block]
    cid = cid[pop[pop>=min_block].argsort()[::-1]]        
    cl = np.r_[[(cc == pc) for pc in cid]]
    return cl,np.ones(T.shape[0],bool)-cl.sum(axis=0)
    

def recursive_cluster(_M,min_block=10,max_block=150,levels=30,screen=True):
    levels+=levels%2
    # find data range:
    
    M = _M.copy().minimum(_M.copy())
    
    M.data[M.data<0.0] = 0.0
    M.eliminate_zeros()
    
    M.data = -np.log(M.data)
    
    M=M.maximum(M.transpose())
    
    thlv = np.percentile(M.data,np.exp(np.linspace(np.log(1),np.log(99),levels)))
    
    nc,cc = connected_components(M)
    pop = np.bincount(cc)
    single_nodes = np.zeros(M.shape[0],bool)
    for i in np.arange(nc)[pop==1]:
        single_nodes += (cc==i)
    
    print("\n\t%d single nodes\n" % single_nodes.sum())
    
    
    
    T = M.copy()
    groups = []
    for th in thlv[::-1]:
        T = M.copy()
        T.data[T.data>th] = 0.0
        T.eliminate_zeros()
        nc,cc = connected_components(T)
        pop = np.bincount(cc)
        
        
        clusters=set(np.arange(nc)[(pop<max_block) * (pop>=min_block)])
        #clusters-=set(np.unique(cc[single_nodes]))
        
        if len(clusters)==0:
            continue
        
        ## all groups less than max_block and more than min_block
        lg = np.r_[[cc==ii for ii in clusters]].astype(bool)
        
        # if largest group small enough, break
        if pop.max()<max_block:
            break
        
        # else, record them with thresh
        if lg.shape[0]>0:
            groups.append([th,lg])
        
    
    #print("\n\tgroup sizes > min_block at th=",th,": ",pop[pop>=min_block]," i.e. ",len(pop[pop>=min_block]),"groups\n\n")
    if screen:
        print("\n\tgroup sizes > min_block at th=",th,": i.e. ",len(pop[pop>=min_block]),"groups\n\n")
        print("\n\tTotal states in these groups:",lg.sum(),"/",pop.sum(),"\n\n")
        disjoint = (lg.sum(axis=0).astype(bool)==0)

        print("\n\tTotal states not in these groups:",disjoint.sum(),"/",pop.sum(),"\n\n")

        if len(groups)>0:
            print("\n\tGroups at other thresholds:")

        for g in groups:
            print("\n\tth=",g[0]," # states:",g[1].sum()," in groups disjoint to final thresh: ",\
                      g[1].sum(axis=1)[g[1].sum(axis=1)==(g[1]*disjoint).sum(axis=1)].sum())

    return lg,single_nodes # np.r_[[cc==i for i in np.arange(nc)[pop==1]]].sum(axis=0).astype(bool)
    
r_N = N
r_I_states = inter_region.copy()
r_A_states = A_states.copy()
r_B_states = B_states.copy()
r_BF = BF.copy()
r_K = K.copy()
r_B = B.copy()
r_D = D.copy()

# Kemeny test on initial regions first

- Look for exit time variance
- Look at accuracy of eigenvalue decomposition in general - tau weight and rho weight
- Test
- done

In [ ]:
_states = B_states.copy()





KAA = K.copy()[_states,:][:,_states]

QfAA = (diags(D[_states])-KAA.copy()).todense()

QAA = (diags(D)-K.copy()).todense()# (diags(np.ravel(KAA.sum(axis=0)).flatten())-KAA.copy()).todense()

NAA = _states.sum()

rho=np.exp(-BF[_states])/np.exp(-BF[_states]).sum()

"""
Gjj = [(1-Pj)K(1-Pj)]^{-1}
sum_j pi_j Gjj = 
pi_j(1-Pj)K(1-Pj)d_i
"""



NAAr = np.arange(NAA)

mfpt = np.zeros(NAA)
pbar=tqdm(total=NAA)
for i in NAAr:
    try:
        vec = spla.solve(QfAA[NAAr!=i,:][:,NAAr!=i].transpose(),np.ones(NAAr.size-1))*rho[i]
    except :
        continue
    mfpt[NAAr!=i] += vec
    pbar.update(1)
pbar.close()


#plt.plot(mfpt[:,:,0].flatten(),

plt.semilogy(mfpt)



In [ ]:
#riPI.K.rPI = (riPI.K.rPI).T
fN = 100
BF = np.random.uniform(size=fN)
fQ = np.exp(-.3-np.random.uniform(size=(fN,fN)))
fQ = np.diag(np.exp(-BF/2.0))@(fQ+fQ.T)@np.diag(np.exp(BF/2.0))
fQ -= np.diag(fQ.sum(axis=0))

kemeny = np.zeros(fN)

nr = np.arange(fN)
plt.figure(figsize=(8,6),dpi=100)

for rr in [0.5,10.0]:
    mfpt = np.zeros(fN)
    dd = np.exp(-1.00-rr*0.001*np.random.uniform(size=fN))
    for i in nr:
        try:
            vec = -spla.solve((fQ-dd)[nr!=i,:][:,nr!=i].transpose(),np.ones(nr.size-1))*np.exp(-BF[i])
        except :
            continue
        mfpt[nr!=i] += vec
    plt.plot(mfpt)

# Just remove single nodes

In [ ]:
#lg,sn = recursive_cluster((B@diags(D))[inter_region,:][:,inter_region],min_block=2,max_block=100,screen=True)
rm_reg = np.zeros(N,bool)
#lg = recursive_cluster((B@diags(D))[inter_region,:][:,inter_region],min_block=2,max_block=100,screen=True)[0]

#print(lg.shape)
#rm_reg[inter_region] = lg.sum(axis=0).astype(bool)
"""
QQ = (B@diags(D)).minimum((B@diags(D)).transpose())
QQ.eliminate_zeros()

th = np.percentile(QQ.data,50)

QQ.data[QQ.data<th] = 0.0
QQ.eliminate_zeros()

nc,cc=connected_components(QQ)
print(nc,np.bincount(cc))

for i in np.arange(nc):
    print(((cc==i)@A_states).sum(),((cc==i)@B_states).sum())
"""

QQ = (B@diags(D))[inter_region,:][:,inter_region]
nc,cc=connected_components(QQ)
pop = np.bincount(cc)

rm_reg[inter_region] = cc!=pop.argmax()

#
#r_A_states = A_states

#disjoint = (lg.sum(axis=1)-lg.dot(1-A_states.astype(int)-B_states.astype(int)))==0
#rm_reg = lg[disjoint,:].sum(axis=0).astype(bool)

rB,rD,rQ,rN,retries = gt.gt_seq(N=N,rm_reg=rm_reg,B=B,D=D,trmb=10,retK=True,Ndense=10,screen=True)
r_A_states=A_states[~rm_reg].copy()
r_B_states=B_states[~rm_reg].copy()
r_inter_region = inter_region[~rm_reg]
r_BF = BF[~rm_reg]


lg = recursive_cluster((rB@diags(rD))[r_inter_region,:][:,r_inter_region],min_block=2,max_block=100,screen=True)[0]
rm_reg = np.zeros(rN,bool)
#print(lg.shape)
rm_reg[r_inter_region] = lg.sum(axis=0).astype(bool)
GT_Q = gt.gt_seq(N=rN,rm_reg=rm_reg,B=rB,D=rD,trmb=10,retK=True,Ndense=10,screen=True)[2]

r_A_states=r_A_states[~rm_reg].copy()
r_B_states=r_B_states[~rm_reg].copy()
r_inter_region = r_inter_region[~rm_reg]
r_BF = r_BF[~rm_reg]



print(GT_Q.shape,r_A_states.shape,rD.shape)

In [ ]:


fig,axs = plt.subplots(4,2,figsize=(12,18),dpi=100,sharey='row')



for k in range(2):

    ax = axs[0,k]
    axa = axs[1,k]
    sax = axs[2,k]
    eax = axs[3,k]
    
    
    ni = [A_states.sum(),B_states.sum()]
    
    ax.set_title(r"%s$\to$%s (from %d$\to$%d states)" % (names[k],names[1-k],ni[k].sum(),ni[1-k].sum()))
    
    sel = [~B_states,~A_states][k]
    r_sel = [~r_B_states,~r_A_states][k]

    ni = [A_states,B_states][k]
    r_ni = [r_A_states,r_B_states][k]
    
    
    ico_D = diags(D[sel])
    ico_K = K[sel,:][:,sel].copy()
    ico_K.eliminate_zeros()
    
    
    rho = np.exp(-BF) * [A_states,B_states][k]
    rho = rho[sel] / rho[sel].sum()
    
    #rho[rho.argmax()] =  1.0
    #rho[np.arange(rho.size)!=rho.argmax()] = 0.0
    
    r_rho = np.exp(-r_BF) * [r_A_states,r_B_states][k]
    r_rho = r_rho[r_sel] / r_rho[r_sel].sum()
    #r_rho[r_rho.argmax()] =  1.0
    #r_rho[np.arange(r_rho.size)!=r_rho.argmax()] = 0.0
    
    
    print("   tau:",spsolve(ico_D-ico_K,rho).sum())
    gt_tau = spsolve(csr_matrix(GT_Q[r_sel,:][:,r_sel]),r_rho).sum()
    print("GT_tau:",gt_tau)
    

    nu,v,w = spla.eig((ico_D-ico_K).todense(),left=True)
    dp = np.sqrt(np.diagonal(w.T.dot(v))).real
    v = (v.real.dot(np.diag(1.0/dp))).T
    vm = v@rho
    w = (w.real.dot(np.diag(1.0/dp)))
    wm=w.sum(axis=0)
    nu = nu.real
    tau = (wm*vm/nu).sum()
    
    cond = nu.max()/nu.min()
    
    ta = np.logspace(-8,2.0,200)*gt_tau

    
    print("   tau:",tau)
    #p_raw = (v*w*nu)@np.exp(-np.outer(nu,ta))*tau
    p_raw = 1.0-(wm*vm)@np.exp(-np.outer(nu,ta))
    ax.plot(ta/tau,p_raw,'k-',label=r"Full, N=%d, $r_\tau$=%2.2g" % (nu.size,1.0),lw=4)
    
    p_raw = (wm*nu*vm)@np.exp(-np.outer(nu,ta))*tau
    axa.plot(ta/tau,p_raw,'k-',label=r"Full, N=%d, $r_\tau$=%2.2g" % (nu.size,1.0),lw=4)
    
    
    sax.plot((nu*tau)[nu.argsort()],(wm*vm)[nu.argsort()],'s',label=r"Full, N=%d" % nu.size)
    eax.plot((nu*tau)[nu.argsort()],(wm*vm/nu)[nu.argsort()]/tau,'s',label=r"Full, N=%d" % nu.size)
   
    
    itau = (wm/nu)@v
    
    
    kF = (1.0/itau[rho>0.0]).dot(rho[rho>0.0])
    
    gt_tau = spsolve(csr_matrix(GT_Q[r_sel,:][:,r_sel]),r_rho).sum()
    print("GT_tau:",gt_tau)
    nu,v,w = spla.eig(GT_Q[r_sel,:][:,r_sel].todense(),left=True)
    dp = np.sqrt(np.diagonal(w.T.dot(v))).real
    v = (v.real.dot(np.diag(1.0/dp))).T.dot(r_rho)
    w = (w.real.dot(np.diag(1.0/dp))).sum(axis=0)
    
    nu = nu.real
    tau = (v*w/nu).sum()
    rcond = nu.max()/nu.min()
    print("gt_tau, rcond:",tau,rcond)
    
    r_p_raw = 1.0-(v*w)@np.exp(-np.outer(nu,ta))
    ax.plot(ta/tau,r_p_raw,'r--',label=r"GT, N=%d, $r_\tau$=%2.2g" % (nu.size,1.0),lw=4)
    
    r_p_raw = (v*w*nu)@np.exp(-np.outer(nu,ta))*tau
    axa.plot(ta/tau,r_p_raw,'r--',label=r"GT, N=%d, $r_\tau$=%2.2g" % (nu.size,1.0),lw=4)
    
    
    sax.plot((nu*tau)[nu.argsort()],(v*w)[nu.argsort()],'o',label=r"GT, N=%d" % nu.size)
    eax.plot((nu*tau)[nu.argsort()],(v*w/nu)[nu.argsort()]/tau,'o',label=r"GT, N=%d" % nu.size)
    
    ax.plot(ta/tau,1-np.exp(-ta/tau),'g-.',label=r"$\exp(-\tau/\langle\tau\rangle)$",lw=4)
    #ax.plot(ta/tau,1-np.exp(-ta*kF),'b-.',label=r"$\exp(-k^F\tau)$",lw=4)
    
    
    axa.plot(ta/tau,np.exp(-ta/tau),'g-.',label=r"$\exp(-\tau/\langle\tau\rangle)$",lw=4)
    #axa.plot(ta/tau,np.exp(-ta*kF)*tau*kF,'b--',label=r"$\exp(-k^F\tau)$",lw=4)
    
    
    sax.plot(np.r_[[0.99,1.0,1.01]],np.r_[[1.0e-9,1.0,1.0e-9]],'D',label=r"$1/\langle\tau\rangle$")
    #sax.plot(np.r_[[0.99,1.0,1.01]]*kF*tau,np.r_[[1.0e-9,1.0,1.0e-9]],'s',label=r"$k^F$")

    

    
    
    ax.set_xlabel(r"Time $\tau/\langle\tau\rangle$")
    sax.set_xlabel(r"Frequency $\nu\langle\tau\rangle$")
    if k==0:
        #axa.set_ylabel(r"$\langle\tau\rangle({\rm d}/{\rm d\tau}){P}(\tau/\langle\tau\rangle)$")
        ax.set_ylabel(r"CDF ${C}(\tau/\langle\tau\rangle)$")
        axa.set_ylabel(r"PDF $\langle\tau\rangle{P}(\tau/\langle\tau\rangle)$")
        sax.set_ylabel(r"Mode Weight in $1-{C}(\tau/\langle\tau\rangle)$")
        eax.set_ylabel(r"Mode Weight in $\langle\tau\rangle$")

    ax.set_yscale("log")
    axa.set_yscale("log")
    sax.set_yscale("log")
    
    eax.set_yscale("log")
    
    ax.set_ylim(0.001,1.1)
    axa.set_ylim(0.001,10.)
    ax.legend()
    eax.legend()
    axa.legend()
    ax.set_xscale("log")
    axa.set_xscale("log")
    sax.set_xscale("log")
    eax.set_xscale("log")
    sax.set_ylim(0.00001,2.0)
    eax.set_ylim(0.00001,2.0)
    sax.legend()

plt.tight_layout()
    

In [ ]:


#----------------------------------


for rec in range(1):
    rm_reg = np.zeros(r_N,bool)
    
    # [:,r_I_states][r_I_states,:]
    lg,sn = recursive_cluster(r_K,min_block=10,max_block=100,screen=False)
    
    rm_reg[r_I_states] = lg.sum(axis=0)
    print(rm_reg.sum())
    
    
    target = B_states+~r_I_states
    plt.semilogy(spsolve((diags(r_D)-r_K)[~target,:][:,~target].transpose(),np.ones((~target).sum()))[r_I_states[~target]],'o')
    #print(spsolve((diags(r_D)-r_K)[~A_states,:][:,~A_states].transpose(),np.ones((~A_states).sum()))[B_states[~A_states]])
    
    #plt.semilogy(spsolve((diags(r_D)-r_K)[~target,:][:,~target].transpose(),np.ones((~target).sum()))[rm_reg[~target]],'o')
    
    break
    
    
    r_B,r_D,r_K,r_N,retries = gt.gt_seq(N=r_N,rm_reg=rm_reg,B=r_B,D=r_D,trmb=10,retK=True,Ndense=10,screen=True)
    
    
    
    
    #rm_reg[r_I_states.nonzero()[0][np.r_[[ll.nonzero()[0][r_BF[r_I_states][ll].argmin()] for ll in lg]].astype(int)]] = False
    #print("Removing %d states" % rm_reg.sum())
    
    
    """
    work out times...
    """
    
    rm_reg = ~rm_reg
    
    #rm_reg[A_states+B_states] = False
    r_B,r_D,r_K,r_N,retries = gt.gt_seq(N=r_N,rm_reg=rm_reg,B=r_B,D=r_D,trmb=10,retK=True,Ndense=10,screen=True)
    
    #target = (A_states+B_states)[~rm_reg]
    
    print(spsolve(r_K[~A_states[~rm_reg],:][:,~A_states[~rm_reg]].transpose(),np.ones((~A_states[~rm_reg]).sum()))[B_states])
    print(spsolve(r_K[~A_states[~rm_reg],:][:,~A_states[~rm_reg]].transpose(),np.ones((~A_states[~rm_reg]).sum()))[B_states])
    
    
    
    break
    
    
    
    r_B,r_D,r_K,r_N,retries = gt.gt_seq(N=r_N,rm_reg=rm_reg,B=r_B,D=r_D,trmb=10,retK=True,Ndense=10,screen=False)
    
    
    r_K = r_B@diags(r_D)
    r_A_states = r_A_states[~rm_reg]
    r_B_states = r_B_states[~rm_reg]
    r_I_states = r_I_states[~rm_reg]
    r_BF = r_BF[~rm_reg]

GT_Q = diags(r_D) - r_K
print(GT_Q.shape,r_B_states.shape,r_A_states.sum(),r_B_states.sum())

